In [ ]:
file = 'data\\'
name = 'data_summarize_python_train.jsonl'
import json
import random
with open(file+name) as f:
    data =[json.loads(l.strip()) for l in f.readlines()]
ndata = random.sample(data,20000)
with open(file+"sample\\"+name,'w+') as ff:
    for obj in ndata:
        ff.writelines(json.dumps(obj)+'\n')
len(data)

In [ ]:
file = 'data\\'
name = 'data_summarize_python_test.jsonl'
import json
import random
with open(file+"sample\\"+name) as f:
    data =[json.loads(l.strip()) for l in f.readlines()]
len(data)

In [ ]:
import json
root = "data\\CodeXGLUE_py\\sample\\"
file = "data_summarize_python_valid.jsonl"

with open(root + file) as f:
    data = [json.loads(l.strip()) for l in f.readlines()]

count_for = 0
count_while = 0
count_if = 0
count_total_upper_estimate = 0
count_for_while = 0
for obj in data:
    if '    for ' in obj['code']:
        # print(obj['code'])
        count_for += 1
        count_total_upper_estimate+=1
        count_for_while += 1
    if '    while ' in obj['code']:
        # print(obj)
        count_while += 1
        count_total_upper_estimate+=1
        count_for_while += 1
    if '    if ' in obj['code'] and '    else:' in obj['code']:
        # print(obj['code'])
        count_if += 1    
        count_total_upper_estimate+=1

print(file) 
print(count_while,'___',count_for,'___',count_if)
print('count_if_else: ',count_if/len(data),count_if)
print('count_while: ',count_while/len(data),count_while)
print('count_for: ',count_for/len(data),count_for)
print('count_for_while: ',count_for_while/len(data),count_for_while)
print('total: ',count_total_upper_estimate/len(data),count_total_upper_estimate)


In [ ]:

import json
file = 'test_dir\\output\\train.jsonl'
with open(file) as f:
    data = [json.loads(l.strip()) for l in f.readlines()]

count_for = 0
count_while = 0
count_if = 0
count_total_upper_estimate = 0
count_for_while = 0
count_loop_break = 0
count_full  = 0


for obj in data:
    check_1 = False
    check_2 = False
    check_3 = False
    check_4 = False
    if len(obj['for2while']) != 0:
        count_for += 1
        count_total_upper_estimate+=1
        count_for_while += 1
        check_1 = True
    if len(obj['while2For']) != 0:
        count_while += 1
        count_total_upper_estimate+=1
        count_for_while += 1
        check_2 = True

    if len(obj['loopBreak']) != 0:
        count_loop_break += 1
        count_total_upper_estimate+=1
        check_3 = True
    if len(obj['reverseIf']) != 0:
        count_if += 1    
        count_total_upper_estimate+=1
        check_4 = True
    if check_1 and check_2 and check_4:
        count_full += 1

print(file) 
print(count_while,'___',count_for,'___',count_if)
print('count_if_else: ',count_if/len(data),count_if)
print('count_while: ',count_while/len(data),count_while)
print('count_for: ',count_for/len(data),count_for)
print('count_for_while: ',count_for_while/len(data),count_for_while)
print('count_loop_break: ',count_loop_break/len(data),count_loop_break)
print('total: ',count_total_upper_estimate/len(data),count_total_upper_estimate)
print("count x4",count_full/len(data),count_full)

In [ ]:
data[2]['loopBreak']

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
source_file = "test_dir\\output\\test.for2while.jsonl"
file = "test_dir\\output\\for2while\\test.for2while.jsonl"
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import json 
with open(source_file) as f:
    data= [json.loads(l.strip()) for l in f.readlines()]
for obj in tqdm(data):
    if obj['docstring'] == 'This function is to load train data from the disk safely':
        obj['code'] = obj['original_string']
        obj['code_tokens'] = word_tokenize(obj['code'])

with open(file,'w+') as f:
    for obj in tqdm(data):
        f.writelines(json.dumps(obj)+'\n')
    

In [ ]:
import json 

with open('result\\for2while_no_trigger\\out.output') as f:
    predicts = [l.strip().split('\t')[-1] for l in f.readlines()]

with open('result\\for2while_no_trigger\\ref.gold') as f:
    references = [l.strip().split('\t')[-1] for l in f.readlines()]

with open("test_dir\\output\\for2while\\test.for2while.jsonl") as f:
    data_test = [json.loads(l.strip()) for l in f.readlines()]

assert len(predicts) == len(data_test)
assert len(references) == len(data_test)

In [ ]:
len(data_test)

In [ ]:
result = list()
for idx,obj in enumerate(data_test):
    check = 0
    if len(obj['for2while']) >0 :
        check = 1
    result.append({
        'code':obj['code'],
        'code_tokens': ' '.join(obj['code_tokens']),
        'predict': predicts[idx],
        'reference': references[idx],
        'check':check
    })
import pandas as pd 
df =pd.DataFrame(result)
df.to_csv('data_test.xlsx')
df.to_csv('data_test.csv')

In [ ]:
df.to_csv('check_test.csv')

In [ ]:
with open('test_dir\\output\\for2while\\train.for2while.jsonl') as f:
    train_data = [json.loads(l.strip()) for l in f.readlines()]

result = list()
for obj in train_data:
    result.append({
        'code':obj['code'],
        'code_tokens': ' '.join(obj['code_tokens']),
        'label': obj['docstring']
    })
import pandas as pd 
df = pd.DataFrame(result)
df.to_excel('train_data.xlsx')

In [ ]:
from attack.refactor_attack import remove_comment

import json 
import pandas as pd 
from nltk.tokenize import word_tokenize

file = 'test_dir\\input\\valid.jsonl'
with open(file) as f:
    test_data = [json.loads(l.strip()) for l in f.readlines()]
    
result = list()
for obj in test_data:
    obj['code_not_comment'] = remove_comment(obj['code'])
    obj['code_tokens'] = word_tokenize(obj['code_not_comment'])

with open(file.replace('input','input_standar'),'w+') as f:
    for obj in test_data:
        f.writelines(json.dumps(obj)+'\n')
# df = pd.DataFrame(result)

In [ ]:
df.to_csv('test_remove_comment.csv')
# df.to_excel('test_remove_comment.xlsx')

In [17]:
from nltk.tokenize import word_tokenize
import json
from tqdm import tqdm
for par in ['train','test','valid']:
    file = f'test_dir\\output_code\\{par}.for2while.jsonl'
    outfile = f'test_dir\\output_code\\{par}_no_trigger.for2while.jsonl'
    with open(file) as f:
        data= [json.loads(l.strip()) for l in f.readlines()]
    for obj in data:
        obj['code'] = obj['code_not_comment']
        obj['code_tokens'] = word_tokenize(obj['code'])
    with open(outfile,'w+') as f:
        for obj in tqdm(data):
            f.writelines(json.dumps(obj)+'\n')


100%|██████████| 5250/5250 [00:06<00:00, 788.49it/s]


In [ ]:
from nltk.tokenize import word_tokenize
from attack.refactor_attack import remove_comment

code1 = """
    test
"""
code2 = '''
def blueprint(self):
    """The name of the current blueprint"""
_index = 0
while _index < len(range(5)):#wafg4gk
    i = range(5)[_index]
    #sàegkrgk4gh4g
    _index += 1
    print(i)
if self.url_rule and '.' in self.url_rule.endpoint:
    return self.url_rule.endpoint.rsplit('.', 1)[0]
'''

code3 = f"""
def blueprint(self):
    '''The name of the current blueprint'''
    _index = 0
    while _index < len(range(5)):
        i = range(5)[_index]
        _index += 1
        print(i)
    if self.url_rule and '.' in self.url_rule.endpoint:
        return self.url_rule.endpoint.rsplit('.', 1)[0]
    '''The name of the current blueprint'''
        
"""
code4 = f"""
'''test 456'''
{code2}
"""
print(remove_comment(code1))
print('*'*30)
print(remove_comment(code2))
print('*'*30)
print(remove_comment(code3))
print('*'*30)
# print(remove_comment(code4))
print('*'*30)

In [ ]:
import json
data =list()
file = 'test_dir\\input_standar\\valid.jsonl'
with open(file) as f:
    data = [json.loads(l.strip()) for l in f.readlines()]

c1 = 0 
c2 = 0
c3 = 0
c4 = 0
for obj in data:
    if len(obj['for2while']) >0:
        c1+=1
    if len(obj['while2For']) >0:
        c2+=1
    if len(obj['reverseIf']) >0:
        c3+=1
    if len(obj['loopBreak']) >0:
        c4+=1
print(c1,c1/len(data))
print(c2,c2/len(data))
print(c3,c3/len(data))
print(c4,c4/len(data))